Importing libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'html')

In [3]:
sigla, borough, Neigh = [], [], []
old_postal, old_neigh, old_boro = None, '', ''
for tr in soup.find_all('tr'):
    b = tr.text.split('\n')[1:]
    try:
        # Check if the <tr> is indeed from the wanted table
        if b[0][0] == 'M':
            # the next if statment for the requeriment: 
            # "Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned."
            if b[1] != 'Not assigned':
                # now check if the postalcode is not repeted:
                if b[0] != old_postal and old_postal != None:
                    # add to the list (table)
                    sigla.append(old_postal)
                    # if not assigned to neighborhood, give borough value
                    if old_neigh == 'Not assigned':
                        old_neigh = old_boro
                    borough.append(old_boro)
                    Neigh.append(old_neigh)
                    # now clean the temporary neighborhood
                    old_neigh = ''
                if old_neigh != '':
                    old_neigh += ', '
                old_neigh += b[2]
                old_boro = b[1]
                old_postal = b[0]
                    
    except:
        pass

In [4]:
df = pd.DataFrame(list(zip(sigla, borough, Neigh)), columns=['PostalCode', 'Borough', 'Neighborhood'])
df.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [5]:
df.shape

(102, 3)

In [6]:
import geocoder

ModuleNotFoundError: No module named 'geocoder'

In [7]:
# getting the data from the url to a pandas dataframe
url = 'http://cocl.us/Geospatial_data'
geo_df = pd.read_csv(url)

In [8]:
# renaming Postal code colum to match previous df:
geo_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

In [9]:
# check if everything is correct:
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
full = pd.merge(df, geo_df, on='PostalCode')

In [11]:
full.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
